In [1]:
import h5py

file_name = '../carbonara_embeddings.h5'
f = h5py.File(file_name, 'r')

In [2]:
with open('../carbonara_sequence_names.txt', 'r') as file:
    rows = [row.strip().split(" ")[0] for row in file.readlines()]

rows = [row for row in rows if "unrelaxed_alphafold2_ptm_model" not in row] # Filter data we can't use

In [3]:
import pandas as pd

df = pd.read_csv("../train.csv")
protein_sequences = df["protein_sequence"].values
tm = df["tm"].values
print(len(tm))

31390


In [4]:
import numpy as np

input_features_z = []
input_features_p = []
output_tm = []
ids = []
sequences = []
for i in range(0,len(rows),5):
    cur_input_z = np.array(f[f"{rows[i]}/z"])
    cur_input_p = np.array(f[f"{rows[i]}/p"])
    for j in range(1,5):
        cur_input_z = np.concatenate((cur_input_z, np.array(f[f"{rows[i + j]}/z"])), axis=1)
        cur_input_p = np.concatenate((cur_input_p, np.array(f[f"{rows[i + j]}/p"])), axis=1)
    input_features_z.append(cur_input_z)
    input_features_p.append(cur_input_p)
    index = int(rows[i].split("_")[0])
    ids.append(index)
    sequences.append(protein_sequences[index])
    output_tm.append(tm[index])
    if(i%1000==0):
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000


In [5]:
import numpy as np
import json

plddt = []
for i in range(0,len(rows),5):
    cur_plddt = np.array([])
    for j in range(5):
        file_name = f'../results/{rows[i+j].replace("unrelaxed", "scores")}.json'
        with open(file_name, 'r') as file:
            data = json.load(file)
        cur_pldtt = np.concatenate((cur_plddt, np.array(data["plddt"])), axis=0)
    plddt.append(cur_pldtt)
    if(i%1000==0):
        print(i)

0
1000


2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000


In [6]:
with h5py.File('../carbonara_compressed.h5', 'w') as f:
    # Store input_features as a dataset
    for i, feature in enumerate(input_features_z):
        f.create_dataset(f'carbonara_z_{i}', data=feature)

    for i, feature in enumerate(input_features_p):
        f.create_dataset(f'carbonara_p_{i}', data=feature)

    for i, feature in enumerate(plddt):
        f.create_dataset(f'plddt_{i}', data=feature)
    
    # Store output_tm as a dataset
    f.create_dataset('output_tm', data=output_tm)
    f.create_dataset('sequences', data=sequences)
    f.create_dataset('ids', data=ids)